# Tornado Heuristics

In [34]:
import pandas as pd

tornado_transactions = pd.read_csv('../data/tornado_trx_transfers.csv', index_col=[0])
intra_set_asset_transfers = pd.read_csv('../data/all_intra_transfers.csv', index_col=[0], low_memory=False)

In [35]:
tornado_eth = tornado_transactions[tornado_transactions['tokenName']=='Ether'].copy()

# Find deposit addresses for ETH pools
deposit_eth = tornado_eth[tornado_eth['functionName']=='deposit(bytes32 _id)']['from'].tolist()

# Find withdrawal addresses for ETH pools
withdraw_eth = tornado_eth[tornado_eth['functionName']=='withdraw(bytes _proof, bytes32 _root, bytes32 _nullifierHash, address _recipient, address _relayer, uint256 _fee, uint256 _refund)']['from'].tolist()

tornado_eth['transaction_type'] = tornado_eth['functionName'].apply(lambda x: 'deposit' if x == 'deposit(bytes32 _id)' else 'withdraw' if x == 'withdraw(bytes _proof, bytes32 _root, bytes32 _nullifierHash, address _recipient, address _relayer, uint256 _fee, uint256 _refund)' else None)

In [36]:
tornado_token = tornado_transactions[tornado_transactions['tokenName']!='Ether'].copy()

# Find deposit addresses for other token pools
tornado_token['transaction_type'] = tornado_token['isSet'].apply(lambda x: 'withdraw' if x == 'to' else 'deposit' if x == 'from' else None)

# Find withdrawal addresses for other token pools
withdraw_token = tornado_token[tornado_token['isSet']=='to']['to'].tolist()
deposit_token = tornado_token[tornado_token['isSet']=='from']['from'].tolist()

## Heuristic 1 - Address Match
No user deposited to and withdrew from a Tornado pool with the same address. The code below is simplified, as it does not check by pool.

In [37]:
address_match_eth = list(set(withdraw_eth) & set(deposit_eth))
print(address_match_eth)

address_match_token = list(set(withdraw_token) & set(deposit_token))
print(address_match_token)

[]
[]


## Heuristic 2 - Unique Gas Price

There are no very specific gas prices with a count above 1.

In [38]:
tornado_concat = pd.concat([tornado_eth, tornado_token], ignore_index=True)

gas_prices = tornado_concat.groupby(['gasPrice']).size().reset_index(name='count')[['gasPrice', 'count']]

filtered_gas_prices = gas_prices.query('count >= 2')
filtered_gas_prices

,gasPrice,count
0,1000000000,3
1,1010000000,2
11,11000000000,7
12,11200000000,2
14,13000000000,7
15,15100000000,2
18,21000000000,3
25,25000000000,2
26,26000000000,3
28,33000000000,2


## Heuristic 3 - Linked Addresses
No interaction between deposit and withdrawal addresses for both the withdrawal and the deposit addresses.

In [40]:
# take intra-set asset transfer dataframe
# check if there are transfers where both the from and to are in the subsets
filtered_df = intra_set_asset_transfers[
    ((intra_set_asset_transfers['from'].isin(withdraw_eth)) & (intra_set_asset_transfers['to'].isin(deposit_eth))) |
    ((intra_set_asset_transfers['from'].isin(deposit_eth)) & (intra_set_asset_transfers['to'].isin(withdraw_eth)))
]
filtered_df

,from,to,timeStamp,nonce,value,gasPrice,input,gasUsed,functionName,chainName,contractAddress,tokenName,tokenType,tokenID
hash,,,,,,,,,,,,,,


In [41]:
filtered_df = intra_set_asset_transfers[
    ((intra_set_asset_transfers['from'].isin(withdraw_token)) & (intra_set_asset_transfers['to'].isin(deposit_token))) |
    ((intra_set_asset_transfers['from'].isin(deposit_token)) & (intra_set_asset_transfers['to'].isin(withdraw_token)))
]
filtered_df

,from,to,timeStamp,nonce,value,gasPrice,input,gasUsed,functionName,chainName,contractAddress,tokenName,tokenType,tokenID
hash,,,,,,,,,,,,,,


## Heuristic 4 - Multiple Denomination
No address withdrew from a specific pool more than once, therefore, this heuristic is not applicable.

## Heuristic 5 - Torn Minting